In [2]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

init_db(2_000_000)

execute_query("SHOW TABLES", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Products table")
execute_query("DESC products", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Brands table")
execute_query("DESC availability", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Brands table")
execute_query("DESC brands", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Categories table")
execute_query("DESC categories", database=get_database_name(), print_as_df=True, show_metrics=False)

print("Colors table")
execute_query("DESC colors", database=get_database_name(), print_as_df=True, show_metrics=False)

Created empty database
Created empty tables
Starting to insert data

Starting insert into 'products' (2000000 rows)...
Finished inserting 2000000 rows into 'products' in 46.73s.


,Tables_in_test_database
0,availability
1,brands
2,categories
3,colors
4,products


Products table


,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,,None,
2,description,text,YES,,None,
3,brand_id,int,YES,MUL,None,
4,category_id,int,YES,MUL,None,
5,price,float,YES,,None,
6,currency,varchar(10),YES,,None,
7,stock,int,YES,,None,
8,ean,varchar(50),YES,,None,
9,color_id,int,YES,MUL,None,


Brands table


...

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,UNI,None,


Categories table


,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,UNI,None,


Colors table


,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(100),YES,UNI,None,


I just quickly want to show the difference a single column index can make in a search and when it does not affect anything

First lets compare how it looks if you search for a spesfic name and compare that to how it is different if we have the column indexd or not

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
SELECT * FROM products WHERE size = 'Large';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)


In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_size ON products(size);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE size = 'Large';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

This is acutally not that huge or a difference i expected so lets try on something that have lets hits.
If there is around 200.000 or under records then it seems that the performance is almost the same where there would be a slight increase in performance if there were a total of 2m records.

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
SELECT * FROM products WHERE name = 'Pro Charger Tablet Brush Go 360';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_name ON products(name);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE name = 'Pro Charger Tablet Brush Go 360';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

Now this is a huge difference. Even when the data size increases you hit the record in no time since there is only one row affected. 
So i need to keep in mind that a index does not help a lot if you query for something that will hit a lot.
I also think that MySQL will in its execution planner evaluate if a full table scan is more effecient or not.

But this would mean to me atleast that you can always create a index on a column and it will either be faster or as fast as a full table scan never slower unless there is something funky with the execution planner.

Lets try to drive this point home, so lets create a index where the query would select almost everything of the DB and then see if it uses the index or if it uses a full scan

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_price ON products(price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE price > 50;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

Yeah it seems like the MySQL execution planner thinks that 50% of the rows will be affected so it wont use the index. Now lets check when it will hit it

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

query = """
CREATE INDEX idx_price ON products(price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE price > 850;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name(), output=False)

clear_mysql_cache()

query = """
CREATE INDEX idx_price ON products(price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT * FROM products WHERE price > 852;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN "+ query, database=get_database_name(), print_as_df=True, show_metrics=False)

It seems like it first happened when the execution planer first thought that 100 percent of the rows hit would be affected by the where clause. That seems a bit odd to me, the difference between is between when the price is over 851 then it use a full scan and over 852 then it uses a range scan

In [ ]:
query = """
SELECT avg(price) from products;
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=True)